In [1]:
import sys
import os
sys.path.append("../../TychePlot/")
#%matplotlib inline
import matplotlib.pyplot as plt
from Plot_processing import calc, export_data, calc_plot_list
from PDF import PDF
from OLEDPlot import OLEDPlot
from Filereader import fileToNpArray
import dill
import copy

In [2]:
#Constants
Voltage=1
Current=2
Current_density=3
Luminance=4
Radiance=5
Current_Efficacy=6
Luminous_Efficacy=7
EQE=8
POWER=9

fill="_"
jvl_file_format=OLEDPlot.jvl_file_format_default
spectra_file_format=OLEDPlot.spectral_data_format_default

In [3]:
#Parameters

## rte
multithreaded=True

## files
series_indicator="XXXI"
series_name="Tuebingen_perovskites_1_5pct_curr"
series_sample_qty=8
jvl_subdir="jvl/"
jvl_concat_subdir="jvl-HV/"
spectra_subdir="spectra/"
valid_pixel_file="valid_pixel.bin"
output_file_path="jvl_plot/"

## figure
figscale=0.6
present_width_pt=665.79451
present_HW_ratio=0.82

## plot data
labelList=[
            r"Ir(ppy)\textsubscript{3}",
            r"CsPbBr\textsubscript{2}Cl",
            r"CsPbBr\textsubscript{2}Cl-ZnS",
            r"CsPbBr\textsubscript{2}Cl-ZnS-PVK-1:1",
            r"CsPbBrI\textsubscript{2}",
            r"CsPbBrI\textsubscript{2}-PVK-1:9",
            r"CsPbBrI\textsubscript{2}-PVK-1:4",
            r"CsPbBrI\textsubscript{2}-PVK-1:1",
]

spectra=[
            ('XXXII',2),
            ('XXXII',3),
            ('XXXII',3),
            ('XXXII',3),
            ('XXXII',4),
            ('XXXII',4),
            ('XXXII',4),
            ('XXXII',4),
]

present={
            #"customFontsize":[8,8,4,6,6],
            "fig_width_pt":present_width_pt*figscale,
            "scaleX":(present_width_pt*figscale)/OLEDPlot.fig_width_default_pt,
            "fixedFigWidth":True,
            "HWratio":present_HW_ratio,
            "titleBool":False,
            "filenamePrefix":output_file_path+"presentation/"
}

In [4]:
inputParameters_colorblind={
                    "spectraFile":"",
                    "labels":[],
                    "titleBool":False,
                    "customFontsize":[12,12,8,10,10],
                    "averageMedian":True,
                    "noSweepBackMeasured":True,
                    "averageSweepBack":False,
                    "useTex":True,
                    "filenamePrefix":output_file_path,
                    "pixelsize_mm2":1.8,
                    #"fileFormat":{"separator":"\t", "skiplines":1, "fileEnding":".uil", "commaToPoint":True},
                    #"spectralDataFormat":{"separator":";", "skiplines":82, "fileEnding":".csv", "commaToPoint":True},
                    "showErrorOnlyEvery":2,
                    "showMarkers":True,
                    "iterMarkers":True,
                    "showLines":False,
                    #"spectralDataFormat":{"separator":",", "skiplines":82, "fileEnding":".csv"}
                }

In [5]:
inputParameters={
                    "spectraFile":"",
                    "labels":[],
                    "titleBool":False,
                    "customFontsize":[12,12,8,10,10],
                    "averageMedian":True,
                    "noSweepBackMeasured":True,
                    "averageSweepBack":False,
                    "useTex":True,
                    "filenamePrefix":output_file_path,
                    "pixelsize_mm2":1.8,

                    #"fileFormat":{"separator":"\t", "skiplines":1, "fileEnding":".uil", "commaToPoint":True},
                    #"spectralDataFormat":{"separator":";", "skiplines":82, "fileEnding":".csv", "commaToPoint":True},
                    "showErrorOnlyEvery":2
                    #"spectralDataFormat":{"separator":",", "skiplines":82, "fileEnding":".csv"}
                }

In [6]:
optionalParameters={
                    "customLims":True, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in volt
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    None,#Current_density
                                    None,#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    None,#EQE
                                    None#POWER
                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    [10**-4,10**3],#Current_density
                                    [10**0,10**7],#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    None,#EQE
                                    None#POWER
                                  ]
}
optionalParameters_cur={
                    "customLims":False, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in volt
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    None,#Current_density
                                    None,#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    None,#EQE
                                    None#POWER
                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    [10**-5,10**1],#Current_density
                                    None,#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    None,#EQE
                                    None#POWER
                                  ]
}

In [7]:
desiredPlots_lum=[
                {
                    "xCol":Voltage,
                    "yCol":Current_density,
                    "yCol2":Luminance,
                    "custom":
                    {
                        "injCode":"ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))"
                    }
                },        
                {
                    "xCol":Luminance,
                    "yCol":EQE,
                    "custom":
                    {
                        "xLim":[5*10**-1,10000],
                        "limCol":Luminance,
                        #"xAxisLim":[10**-1,1000],
                        "legLoc":1,
                        #"show":[[False,False],[True,True]],
                    }
                },
                {
                    "xCol":Current_density,
                    "yCol":Luminance,
                    "custom":
                    {
                        "optionalParameters":optionalParameters_cur
                    }
                },
                    {
                    "xCol":Voltage,
                    "yCol":POWER,
                },
]
desiredPlots_dark=[
                {
                    "xCol":Voltage,
                    "yCol":Current_density,
                    #"yCol2":Luminance,
                    "custom":
                    {
                        "injCode":"ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))"
                    }
                },        
                    {
                    "xCol":Voltage,
                    "yCol":POWER,
                },
]

In [8]:
groups=[
        (0,),
        (1,2,3),
        (4,),
        (5,6,7),
]
group_names=[
                "irppy",
                "chlorine-compare",
                "iodine",
                "iodine-HV"
]
group_specific_input=[
                         {
                            "desiredPlots":desiredPlots_lum,
                            "inputParameters":inputParameters,
                            "optionalParameters":optionalParameters,
                        }]+[
                        {
                            "desiredPlots":desiredPlots_lum,
                            "inputParameters":inputParameters,
                            "optionalParameters":optionalParameters,
                        },
]*3

In [9]:
#Dynamic Parameters
name=series_indicator+fill+series_name

In [10]:
filelist=OLEDPlot.generateFileList(series_indicator, subdir=jvl_concat_subdir)
fileList_conc=[]
for device in filelist:
    subList2=[]
    for n in range(0,series_sample_qty):
        sublist=[]
        for pixel in device:
            if int(pixel[-1])==n:
                sublist.append(pixel)
        if sublist != []:
            subList2.append(sublist)
    fileList_conc+=subList2
fileList_conc=[concat_tup for concat_tup in fileList_conc if len(concat_tup)>1]

In [11]:
#fileList_conc

In [12]:
for file_tup in fileList_conc:
    OLEDPlot.concentenate_files(file_tup, fileFormat=OLEDPlot.jvl_file_format_default, subdir=jvl_subdir, previous_subdir=jvl_concat_subdir)

In [13]:
#getting valid_pixels (file or user_input)
try:
    with open(jvl_subdir+valid_pixel_file, "rb") as f:
        valid_info=dill.load(f)
except FileNotFoundError:
    valid_info=OLEDPlot.get_valid_pixel_by_user(series_indicator, samples=series_sample_qty, subdir=jvl_subdir, update_by_existing=True, jvl_file_format={"separator":"\t", "skiplines":1, "fileEnding":".uil", "commaToPoint":True})
    with open(jvl_subdir+valid_pixel_file, "wb") as f:
        dill.dump(valid_info, f)
valid_pixel=valid_info[1]
valid_device=valid_info[0]

In [14]:
#valid_pixel[7]=[False,False,False,True]

In [15]:
#Importing files
#spectra=sorted([spectra_subdir+file for file in os.listdir(spectra_subdir) if spectra_file_format["fileEnding"] == file[-4:]])
OLED_fileList=OLEDPlot.generateFileList(series_indicator, samples=series_sample_qty, subdir=jvl_subdir, truthTable=valid_pixel, update_by_existing=True)
OLED_fileList

[['jvl/XXXII_A_1', 'jvl/XXXII_A_2', 'jvl/XXXII_A_3', 'jvl/XXXII_A_4'],
 ['jvl/XXXII_B_2', 'jvl/XXXII_B_3', 'jvl/XXXII_B_4'],
 ['jvl/XXXII_C_1', 'jvl/XXXII_C_3', 'jvl/XXXII_C_4'],
 ['jvl/XXXII_D_2', 'jvl/XXXII_D_3', 'jvl/XXXII_D_4'],
 ['jvl/XXXII_E_1', 'jvl/XXXII_E_3'],
 ['jvl/XXXII_F_2', 'jvl/XXXII_F_3'],
 ['jvl/XXXII_G_1', 'jvl/XXXII_G_3'],
 ['jvl/XXXII_H_2', 'jvl/XXXII_H_3']]

In [16]:
plot_prop_list=[]
feature="default"
for group,group_name,specific_input in zip(groups,group_names,group_specific_input):
    fileList=[]
    spectra_loc=[]
    labels_loc=[]
    try:
        inputParameters_loc=copy.deepcopy(specific_input["inputParameters"])
    except:
        inputParameters_loc=copy.deepcopy(inputParameters)
    try:
        optionalParameters_loc=copy.deepcopy(specific_input["optionalParameters"])
    except:
        optionalParameters_loc=copy.deepcopy(optionalParameters)
    try:
        desiredPlots_loc=copy.deepcopy(specific_input["desiredPlots"])
    except:
        desiredPlots_loc=copy.deepcopy(desiredPlots)
    for pos in group:
        fileList.append(OLED_fileList[pos])
        if type(spectra[pos]) == tuple:
            spectra_loc.append((spectra_subdir+spectra[pos][0],spectra[pos][1]))
        else:
            spectra_loc.append(spectra_subdir+spectra[pos])
        labels_loc.append(labelList[pos])
    inputParameters_loc["spectraFile"]=spectra_loc
    inputParameters_loc["labels"]=labels_loc
    plot_prop_list.append(
        {
             "name":name+fill+group_name,
             "fileList":fileList,
             "desiredPlots":desiredPlots_loc,
             "present_params":present,
             "inputParameters":inputParameters_loc,
             "plot_class":OLEDPlot,
             "optionalParameters":optionalParameters_loc,
             "feature":feature,
             "multithreaded":multithreaded
        }
        )
results=calc_plot_list(plot_prop_list, multithreaded=multithreaded)

../../TychePlot/OLEDPlot.py:460: UserWarning: Invalid Limits at column 4 with value 5.0E-01
  warnings.warn("Invalid Limits at column "+str(ie)[-1:]+" with value "+str(ie)[45:52])


In [17]:
u=results[0][0][0].expectData[0].getData()[:,0]
i=results[0][0][0].expectData[0].getData()[:,1]
abs(u/i)

array([4.52437507e+06, 4.68470690e+06, 4.76064533e+06, 4.79447226e+06,
       4.83493239e+06, 4.86105485e+06, 4.90926624e+06, 4.93031194e+06,
       4.99219969e+06, 5.00682749e+06, 5.06136910e+06, 5.09698428e+06,
       5.15049091e+06, 5.18710634e+06, 5.23331880e+06, 5.28052805e+06,
       5.38629860e+06, 5.49073439e+06, 5.71306149e+06, 6.38569604e+06,
       1.93888607e-07, 4.43803395e+06, 4.74383302e+06, 4.86203963e+06,
       4.89536164e+06, 4.94413132e+06, 4.93218249e+06, 4.91400491e+06,
       4.89296636e+06, 4.87672717e+06, 4.86144871e+06, 4.83569623e+06,
       4.79328939e+06, 4.76714338e+06, 4.75866757e+06, 4.70108909e+06,
       4.65556121e+06, 4.64671313e+06, 4.59535359e+06, 4.57969268e+06,
       4.51059991e+06, 4.46832278e+06, 4.43682565e+06, 4.42052662e+06,
       4.35987102e+06, 4.26693975e+06, 4.24022506e+06, 4.14189837e+06,
       4.03690888e+06, 3.89798044e+06, 3.65708713e+06, 3.34556443e+06,
       2.97632888e+06, 2.56559767e+06, 2.16388226e+06, 1.77529800e+06,
      